# Shamir Secret Sharing

First, let's import some "helper" functions for dealing with polynomials.

In [ ]:
from helper import *
import matplotlib.pyplot as plt

## Share
Here is our sharing function. It takes as input `s` (the secret), `n` (the number of parties to share amongst), and `t` (the degree of the polynomial used for sharing, so t+1 is the minimum number of parties needed to reconstruct the secret). (You don't need to worry about `p`.) The input `printout` tells the function whether it should print an explanation of what it's doing or not.

In [ ]:
def share(s, n, t, p=0, printout=False):
    if p == 0:
        p = nextprime(s)
        
    # check t
    if t >= n:
        print("t={} must be less than n={}".format(t, n))
        return

    # check s is in field
    if p <= s:
        print("s={} must be less than p={}".format(s, p))
        return
        
    ### the interesting part ###########################
    # pick a random polynomial
    coeffs = []
    for i in range(t):
        coeffs.append(randint(0,p-1))
    # secret is the y-intercept / constant term of the poly
    coeffs.append(s)
    
    shares = []
    for i in range(1,n+1):
        shares.append((i, eval_poly(coeffs, i)))
    #####################################################
        
    if printout:
        # plot the polynomial
        print("The random degree t={} polynomial is".format(t))
        print_poly(coeffs)
        x = linspace(0, n, n+1)
        y = [eval_poly(coeffs, i) for i in x]
        plt.plot(x, y)
    
        # plot the shares
        print()
        print("The shares are points on that polynomial:")
        print(shares)
        x1 = [shares[i][0] for i in range(len(shares))]
        y1 = [shares[i][1] for i in range(len(shares))]
        plt.scatter(x1, y1)
    
        # plot the secret
        plt.scatter(0, s, color='orange')
    
        print()
        print("Here is a visual representation (secret in orange).")
        
    return shares

### Example
For example, say our secret is the number 42. We'd like to share it among 10 parties (n=10), and we'll allow any 4 of those to recover the secret (t+1 can recover, so t=3).

Here's how we would use ("call") the `share` function. Both lines below do the same thing; sometimes adding in the names of the inputs in the function call makes it easier to remember what's what.

In [ ]:
shares = share(42, 10, 3, True)
shares = share(42, n=10, t=3, printout=True)

Now we can distribute these points among our 10 parties!

## Reconstruct
Say 4 of those parties (Alice, Bob, Charlie, and Diane) want to recover the secret. We let t=3, so they should be able to do this (remember, a minimum of t+1 parties is needed). Together, they hold 4 points on the degree-3 polynomial, which uniquely defines it! Remember that they can pool this information to recover the polynomial, which we'll call $f$, and evaluate it at x=0 to get back the secret. This is what the `recon` function does:

In [ ]:
def recon(shares, printout=False):
    # interpolate f and recover secret = f(0)
    f = interpolate(shares)
    s = round(f(0))

    if printout:
        print("The reconstructed secret is: {}".format(s))
    return s

Assuming Alice, Bob, Charlie, and Diane have the points for x=1,2,3,4, respectively, they can you use their shares of 42 (which we created earlier) to recover the secret as follows:

In [ ]:
recon_s = recon([shares[0], shares[1], shares[2], shares[3]], printout=True)

You might notice that the shares we used were numbered starting at 0 instead of 1. This is because Python is a **zero-indexed language**: it starts counting at 0. Another thing to keep in mind is that we could have used any four shares, not just the first four. Play around with the numbers above and see if you still get the same answer when you use different shares.

## Sharing messages
So far we've shared only numbers. This might seem limiting, but the whole world can be represented in numbers! For instance, we can share messages by turning letters into numbers:

| a | b | c | d | e | f | g | h | i | j | k | l | m | n | o | p | q | r | s | t | u | v | w | x | y | z |
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20 | 21 | 22 | 23 | 24 | 25 | 26 |

Then, to send the message "Hello world!", we turn it into a list of numbers: `[8, 5, 12, 12, 15, 23, 15, 18, 12, 4]`. (To keep things simple, we'll ignore spaces, punctuation, and other symbols.) Then we can share each of the numbers in the list, and each party will have a list of numbers that are shares of the message.

The functions `share_msg` and `recon_msg` do exactly this. `share_msg` converts each letter in a message to a number and shares each of those numbers. It outputs a list of shares (points) for each party. Notice that all the points in one party's set have the same x-coordinates!

`recon_msg` takes each party's list of points and reconstructs each letter; then it outputs the recovered message (since we didn't consider spaces, punctuation, etc., those are missing from the reconstructed message).

In [ ]:
def share_msg(msg, n, t, p=29):
    arr = convert_msg(msg)
    shares_by_letter = []
    for x in arr:
        shares_by_letter.append(share(x, n, t, p))
    
    shares_by_party = []
    for i in range(n):
        share_i = [row[i] for row in shares_by_letter]
        shares_by_party.append(share_i)
        print("Party {}'s share is: {}".format(i+1, share_i))
    return shares_by_party

def recon_msg(arr_shares):
    ans = []
    for i in range(len(arr_shares[0])):
        letter_shares = [arr[i] for arr in arr_shares]
        ans.append(recon(letter_shares))
    return nums_to_msg(ans)

### Example
Here is how we share our example message "Hello world!" among 5 parties so that any t+1=3 can recover:

In [ ]:
msg_shares = share_msg("Hello world!", n=5, t=2)

And here is the reconstruction, first using all the shares, then using only 3 of them. As long as we provide at least t+1=3 shares to the reconstruction function, it'll work.

In [ ]:
print(recon_msg(msg_shares))
print(recon_msg([msg_shares[0], msg_shares[1], msg_shares[3]]))

## Now it's your turn!
We're going to break into groups and you'll have a chance to share your own messages among the group! Set `n` equal to the group size (including you!) and then pick whatever threshold `t` you want to determine how many people can reconstruct. Pass out the shares of your secret message (you don't even have to give them all out!) and ask your group members to recover your message. Below is some code you can use, but feel free to play around with it and change parts of it as you like!

Some ideas:
- Set t=(n-2)/2 and pass out all the shares. Then split your group in half and have each half work together to recover the message. Report back with your message – both subgroups should get the same thing!
- Set t=n-2 and pass out all the shares. Ask your group to recover your message without helping. Even without your points, they should still be able to recover the message.

In [ ]:
print("Pass out a lists of points to each of your group members: ")
msg_shares = share_msg("Your message here!", n=5, t=4) # change n, t as you like

In [ ]:
# enter lists of points (shares) to reconstruct like this: 
recon_msg([
    [(1, 39), (1, 57), (1, 60), (1, 12), (1, 55), (1, 55), (1, 37), (1, 19), (1, 38), (1, 24), (1, 38)],
    [(4, 267), (4, 405), (4, 456), (4, 45), (4, 454), (4, 319), (4, 181), (4, 91), (4, 290), (4, 117), (4, 92)],
    [(5, 399), (5, 601), (5, 676), (5, 60), (5, 683), (5, 459), (5, 245), (5, 123), (5, 438), (5, 160), (5, 110)]
])